# Ray Serve - Integration with FastAPI

© 2019-2022, Anyscale. All Rights Reserved


📖 [Back to Table of Contents](./ex_00_tutorial_overview.ipynb)<br>
➡ [Next notebook](./ex_03_model_composition.ipynb) <br>
⬅️ [Previous notebook](./ex_01_model_serving_challenges.ipynb) <br>

Ray Serve integrates well with other common [web serving frameworks](https://docs.ray.io/en/latest/serve/tutorials/web-server-integration.html). 

In this tutorial, we’ll cover how to deploy [XGBoost](https://xgboost.readthedocs.io/en/stable/) with [FastAPI](https://fastapi.tiangolo.com/) and Ray Serve. We'll use a simple XGBboost classifcation model to train, deploy it on Ray Serve , and access it via HTTP request on a FastAPI endpoint. 

FastAPI is a modern, fast (high-performance), web framework for building APIs with Python 3.6+ based on standard Python type hints.

<img src="https://fastapi.tiangolo.com/img/logo-margin/logo-teal.png" width="40%" height="20%"> 

This XGBoost model will be trained to predict the onset of diabetes using the pima-indians-diabetes dataset from the [UCI Machine Learning Repository website](https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv). This small dataset contains several numerical medical variables of eight different features related to diabetes, in addition to one target variable — Outcome. So, we’ll use XGBoost to model and solve a simple prediction problem. This tutorial is derived from our [blog](https://www.anyscale.com/blog/deploying-xgboost-models-with-ray-serve).

Let's see how easy it is!



In [1]:
import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pickle

import ray
import logging
import os
from fastapi import FastAPI, Request
from ray import serve

### Load the data

In [3]:
# Load the data
path = os.path.abspath('data/pima-indians-diabetes.data.csv')
dataset = np.loadtxt(path, delimiter=",")
# split data into X and y
X = dataset[:, 0:8]
y = dataset[:, 8]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=7)

### Define a utility function:
 * Creates XGBoost classifier
 * trains, fits, and saves the model
 * returns a model

In [4]:
def create_and_save_model():
    # Instantiate a model, fit and train
    xgb_model = XGBClassifier(use_label_encoder=False)
    xgb_model.fit(X_train, y_train)

    # saving the model
    path = os.path.abspath('xgb_model.pkl')
    with open(path, 'wb') as f:
        pickle.dump(xgb_model, f)

    return xgb_model

### Create, fit and predict XGBoost model

In [5]:
model = create_and_save_model()
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 74.02%


### Create a Ray Serve Deployment with FastAPI

In [6]:
app = FastAPI()
if ray.is_initialized:
    ray.shutdown()
ray.init(namespace="xgbregressor", logging_level=logging.ERROR)

Python version:,3.8.13
Ray version:,2.0.0rc0
Dashboard:,http://127.0.0.1:8265


In [6]:
@serve.deployment(num_replicas=2, route_prefix="/regressor")
@serve.ingress(app)
class XGBModel:
    def __init__(self):
        # loading the model
        path = os.path.app
        with open("xgb_model.pkl", "rb") as f:
            self.model = pickle.load(f)
        print("Pickled XGBoost model loaded")

    @app.post("/")
    async def predict(self, starlette_request: Request):
        payload = await starlette_request.json()
        print("Worker: received starlette request with data", payload)

        prediction = round(self.model.predict([np.array(list(payload.values()))])[0])
        
        return {"result": prediction}

In [7]:
xgboost_handle = XGBModel.bind()
serve.run(xgboost_handle)

(ServeController pid=71361) INFO 2022-08-08 16:06:43,022 controller 71361 http_state.py:123 - Starting HTTP proxy with name 'SERVE_CONTROLLER_ACTOR:SERVE_PROXY_ACTOR-427538e0d1060913c72672d0da95cbf4d96212d6a5ece44e35f7103e' on node '427538e0d1060913c72672d0da95cbf4d96212d6a5ece44e35f7103e' listening on '127.0.0.1:8000'
(ServeController pid=71361) INFO 2022-08-08 16:06:43,640 controller 71361 deployment_state.py:1232 - Adding 2 replicas to deployment 'XGBModel'.
(HTTPProxyActor pid=71367) INFO:     Started server process [71367]


(ServeReplica:XGBModel pid=71382) Pickled XGBoost model loaded
(ServeReplica:XGBModel pid=71383) Pickled XGBoost model loaded


You are retrieving a sync handle inside an asyncio loop. Try getting Deployment.get_handle(.., sync=False) to get better performance. Learn more at https://docs.ray.io/en/latest/serve/handle-guide.html#sync-and-async-handles


RayServeSyncHandle(deployment='XGBModel')

### Send request to the FastAPI endpoint

In [9]:
import requests

sample_request_inputs = [
    {"Pregnancies": 6,
     "Glucose": 148,
     "BloodPressure": 72,
     "SkinThickness": 35,
     "Insulin": 0,
     "BMI": 33.6,
     "DiabetesPedigree": 0.625,
     "Age": 50,
    },
    {"Pregnancies": 10,
      "Glucose": 168,
      "BloodPressure": 74,
      "SkinThickness": 0,
      "Insulin": 0,
      "BMI": 38.0,
      "DiabetesPedigree": 0.537,
      "Age": 34,
    },
    {"Pregnancies": 10,
     "Glucose": 39,
     "BloodPressure": 80,
     "SkinThickness": 0,
     "Insulin": 0,
     "BMI": 27.1,
     "DiabetesPedigree": 1.441,
     "Age": 57,
     },
     {"Pregnancies": 1,
      "Glucose": 103,
      "BloodPressure": 30,
      "SkinThickness": 38,
      "Insulin": 83,
      "BMI": 43.3,
      "DiabetesPedigree": 0.183,
      "Age": 33,
     }
    ]

In [10]:
# Iterate our requests
for sri in sample_request_inputs:
    response = requests.post("http://localhost:8000/regressor/", json=sri)
    print(response.text)

{"result":1}
{"result":1}
{"result":0}
{"result":0}
(ServeReplica:XGBModel pid=71382) Worker: received starlette request with data {'Pregnancies': 10, 'Glucose': 168, 'BloodPressure': 74, 'SkinThickness': 0, 'Insulin': 0, 'BMI': 38.0, 'DiabetesPedigree': 0.537, 'Age': 34}
(ServeReplica:XGBModel pid=71382) Worker: received starlette request with data {'Pregnancies': 1, 'Glucose': 103, 'BloodPressure': 30, 'SkinThickness': 38, 'Insulin': 83, 'BMI': 43.3, 'DiabetesPedigree': 0.183, 'Age': 33}
(ServeReplica:XGBModel pid=71383) Worker: received starlette request with data {'Pregnancies': 6, 'Glucose': 148, 'BloodPressure': 72, 'SkinThickness': 35, 'Insulin': 0, 'BMI': 33.6, 'DiabetesPedigree': 0.625, 'Age': 50}
(ServeReplica:XGBModel pid=71383) Worker: received starlette request with data {'Pregnancies': 10, 'Glucose': 39, 'BloodPressure': 80, 'SkinThickness': 0, 'Insulin': 0, 'BMI': 27.1, 'DiabetesPedigree': 1.441, 'Age': 57}


(HTTPProxyActor pid=71367) INFO 2022-08-08 16:07:03,290 http_proxy 127.0.0.1 http_proxy.py:315 - POST /regressor 200 10.5ms
(HTTPProxyActor pid=71367) INFO 2022-08-08 16:07:03,305 http_proxy 127.0.0.1 http_proxy.py:315 - POST /regressor 200 10.5ms
(HTTPProxyActor pid=71367) INFO 2022-08-08 16:07:03,311 http_proxy 127.0.0.1 http_proxy.py:315 - POST /regressor 200 3.3ms
(HTTPProxyActor pid=71367) INFO 2022-08-08 16:07:03,316 http_proxy 127.0.0.1 http_proxy.py:315 - POST /regressor 200 3.0ms
(ServeReplica:XGBModel pid=71382) INFO 2022-08-08 16:07:03,304 XGBModel XGBModel#tkDUaO replica.py:482 - HANDLE __call__ OK 5.8ms
(ServeReplica:XGBModel pid=71382) INFO 2022-08-08 16:07:03,316 XGBModel XGBModel#tkDUaO replica.py:482 - HANDLE __call__ OK 1.5ms
(ServeReplica:XGBModel pid=71383) INFO 2022-08-08 16:07:03,288 XGBModel XGBModel#ISxXnN replica.py:482 - HANDLE __call__ OK 6.4ms
(ServeReplica:XGBModel pid=71383) INFO 2022-08-08 16:07:03,310 XGBModel XGBModel#ISxXnN replica.py:482 - HANDLE __ca

In [11]:
ray.shutdown()

📖 [Back to Table of Contents](./ex_00_tutorial_overview.ipynb)<br>
➡ [Next notebook](./ex_04_inference_graphs.ipynb) <br>
⬅️ [Previous notebook](./ex_01_model_serving_challenges.ipynb) <br>